In [1]:
# Importation des librairies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import hex2color

import seaborn as sns
import plotly.express as px

import datetime as dt
import scipy.stats as st
import statsmodels.api as sm
from statsmodels.formula.api import ols

##### Description des emplois salariés en 2018

https://www.insee.fr/fr/statistiques/5366604

Pour utiliser ces fichiers volumineux, il est recommandé d'utiliser un gestionnaire de bases de données ou un logiciel statistique. Les fichiers sont disponibles au format dBase ou csv.

cliquer sur le fichier que vous souhaitez télécharger ;
enregistrer le fichier sur votre disque dur (format zip) ;
décompacter le fichier téléchargé dans un répertoire de votre disque dur.
Chaque répertoire zippé dBase contient, en complément d'une table de données, un fichier varmod.dbf. Il est directement utilisable sous forme électronique pour récupérer les intitulés des variables et de leurs modalités lors de la confection des tableaux.

Le fichier « salariés » décrit les caractéristiques du salarié, de son poste principal ainsi que des données récapitulatives tous postes confondus : rémunérations, heures salariées, total des indemnités de chômage.

Chaque enregistrement est constitué par la consolidation des périodes de travail d'un salarié quels que soient ses employeurs.

Le fichier « salariés » est disponible au format dBase ou csv. Il contient 31 variables et 2 331 440 observations.


Le fichier est issu d'un échantillon au 1/12e de la population salariée. Une variable de pondération égale à 12 est présente dans le fichier Dbase, et le fichier csv est pondéré par défaut par 12.

Avertissement : À partir du millésime 2013, la définition du 1/12e induit que les salariés nés une année paire sont surreprésentés dans l'échantillon (voir Documentation).

In [2]:
df=pd.read_csv('FD_SALAAN_2018.csv', sep=";")
df

C:\Users\vraks\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,TRNNETO,TREFF,A6,A17,A38,CONV_COLL,CPFD,DATDEB,DATFIN,DEPR,...,TYP_EMPLOI,AGE,POND,TRALCHT,CONT_TRAV,CS,AGE_TR,DATDEB_TR,DATFIN_TR,DUREE_TR
0,7,1.0,JU,RU,TZ,2111.0,NaN,1.0,270.0,NaN,...,O,68,12,99,CDI,56.0,71,1.0,9.0,9.0
1,23,1.0,JU,RU,TZ,2111.0,NaN,1.0,360.0,NaN,...,O,37,12,99,CDI,56.0,39,1.0,12.0,12.0
2,8,1.0,JU,RU,TZ,2111.0,NaN,1.0,360.0,NaN,...,O,68,12,99,CDI,56.0,71,1.0,12.0,12.0
3,10,0.0,JU,RU,TZ,2111.0,NaN,1.0,300.0,NaN,...,O,62,12,99,CDI,56.0,63,1.0,10.0,10.0
4,10,1.0,JU,RU,TZ,2111.0,NaN,1.0,360.0,NaN,...,O,59,12,99,CDI,56.0,63,1.0,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331435,13,4.0,GI,IZ,IZ,1979.0,C,1.0,360.0,99,...,O,26,12,99,CDI,56.0,27,1.0,12.0,12.0
2331436,12,3.0,OQ,OQ,PZ,3218.0,C,1.0,360.0,99,...,O,42,12,99,CDI,56.0,43,1.0,12.0,12.0
2331437,7,1.0,GI,GZ,GZ,1090.0,C,1.0,360.0,99,...,A,17,12,99,APP,68.0,19,1.0,12.0,12.0
2331438,16,1.0,GI,GZ,GZ,1090.0,C,1.0,360.0,99,...,O,53,12,99,CDI,63.0,55,1.0,12.0,12.0


In [3]:
# Vérification des valeurs manquantes 
isna_df=df.isna().sum()
isna_df

TRNNETO            0
TREFF          87326
A6             87377
A17            87377
A38               52
CONV_COLL     578265
CPFD          135260
DATDEB         87343
DATFIN         87330
DEPR           29376
DEPT           26547
DOMEMPL        92693
DOMEMPL_EM     87443
DUREE          87326
FILT               0
NBHEUR         88139
NBHEUR_TOT       641
PCS             3579
REGR           29376
REGT           26547
SEXE               0
TYP_EMPLOI         0
AGE                0
POND               0
TRALCHT            0
CONT_TRAV      99530
CS              3579
AGE_TR             0
DATDEB_TR      87343
DATFIN_TR      87330
DUREE_TR       87326
dtype: int64

In [4]:
isna_df.sum()

1987105

In [5]:
# Vérification des doublons
df.duplicated().sum()

44079

In [6]:
# Vérification des outliers
df.describe().round()

,TRNNETO,TREFF,CONV_COLL,DATDEB,DATFIN,DOMEMPL,DOMEMPL_EM,DUREE,FILT,NBHEUR,...,REGT,SEXE,AGE,POND,TRALCHT,CS,AGE_TR,DATDEB_TR,DATFIN_TR,DUREE_TR
count,2331440.0,2244114.0,1753175.0,2244097.0,2244110.0,2238747.0,2243997.0,2244114.0,2331440.0,2243301.0,...,2304893.0,2331440.0,2331440.0,2331440.0,2331440.0,2327861.0,2331440.0,2244097.0,2244110.0,2244114.0
mean,14.0,3.0,1930.0,32.0,334.0,7.0,7.0,296.0,1.0,1406.0,...,47.0,1.0,40.0,12.0,86.0,52.0,43.0,2.0,11.0,10.0
std,5.0,2.0,1769.0,71.0,61.0,3.0,3.0,100.0,0.0,568.0,...,30.0,0.0,13.0,0.0,33.0,10.0,13.0,2.0,2.0,3.0
min,0.0,0.0,2.0,-30.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,12.0,0.0,10.0,0.0,-12.0,1.0,1.0
25%,11.0,1.0,965.0,1.0,360.0,6.0,6.0,240.0,1.0,959.0,...,11.0,1.0,30.0,12.0,99.0,43.0,31.0,1.0,12.0,8.0
50%,14.0,3.0,1597.0,1.0,360.0,9.0,9.0,360.0,1.0,1716.0,...,44.0,1.0,40.0,12.0,99.0,54.0,43.0,1.0,12.0,12.0
75%,18.0,6.0,2372.0,1.0,360.0,9.0,9.0,360.0,1.0,1820.0,...,76.0,2.0,51.0,12.0,99.0,62.0,55.0,1.0,12.0,12.0
max,23.0,6.0,9999.0,360.0,360.0,9.0,9.0,390.0,2.0,2500.0,...,99.0,2.0,127.0,12.0,99.0,69.0,72.0,12.0,12.0,13.0


In [8]:
df_salarie=df.loc[df['AGE'] > 15].copy() 
df_salarie=df_salarie.loc[df_salarie['AGE'] < 70] 

,TRNNETO,TREFF,A6,A17,A38,CONV_COLL,CPFD,DATDEB,DATFIN,DEPR,...,TYP_EMPLOI,AGE,POND,TRALCHT,CONT_TRAV,CS,AGE_TR,DATDEB_TR,DATFIN_TR,DUREE_TR
0,7,1.0,JU,RU,TZ,2111.0,NaN,1.0,270.0,NaN,...,O,68,12,99,CDI,56.0,71,1.0,9.0,9.0
1,23,1.0,JU,RU,TZ,2111.0,NaN,1.0,360.0,NaN,...,O,37,12,99,CDI,56.0,39,1.0,12.0,12.0
2,8,1.0,JU,RU,TZ,2111.0,NaN,1.0,360.0,NaN,...,O,68,12,99,CDI,56.0,71,1.0,12.0,12.0
3,10,0.0,JU,RU,TZ,2111.0,NaN,1.0,300.0,NaN,...,O,62,12,99,CDI,56.0,63,1.0,10.0,10.0
4,10,1.0,JU,RU,TZ,2111.0,NaN,1.0,360.0,NaN,...,O,59,12,99,CDI,56.0,63,1.0,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331435,13,4.0,GI,IZ,IZ,1979.0,C,1.0,360.0,99,...,O,26,12,99,CDI,56.0,27,1.0,12.0,12.0
2331436,12,3.0,OQ,OQ,PZ,3218.0,C,1.0,360.0,99,...,O,42,12,99,CDI,56.0,43,1.0,12.0,12.0
2331437,7,1.0,GI,GZ,GZ,1090.0,C,1.0,360.0,99,...,A,17,12,99,APP,68.0,19,1.0,12.0,12.0
2331438,16,1.0,GI,GZ,GZ,1090.0,C,1.0,360.0,99,...,O,53,12,99,CDI,63.0,55,1.0,12.0,12.0


In [9]:
df_salarie=df_salarie[["A6","TRNNETO","AGE","CPFD","CS","SEXE","CONT_TRAV"]].copy()
df_salarie.rename(columns={'A6': 'Activité_6'}, inplace=True)
df_salarie.rename(columns={'TRNNETO': 'Salaire_net'}, inplace=True)
df_salarie.rename(columns={'AGE': 'Age'}, inplace=True)
df_salarie.rename(columns={'CPFD': 'Temps_travail'}, inplace=True)
df_salarie.rename(columns={'CS': 'Catégorie_SP'}, inplace=True)
df_salarie.rename(columns={'SEXE': 'Sexe'}, inplace=True)
df_salarie.rename(columns={'CONT_TRAV': 'Type_contrat'}, inplace=True)

In [10]:
df_salarie.isna().sum()

Activité_6        85368
Salaire_net           0
Age                   0
Temps_travail    131206
Catégorie_SP       3525
Sexe                  0
Type_contrat      97501
dtype: int64

In [11]:
df_salarie=df_salarie.dropna()
df_salarie = df_salarie.reset_index()
df_salarie=df_salarie.drop(columns=['index']).copy()

,Activité_6,Salaire_net,Age,Temps_travail,Catégorie_SP,Sexe,Type_contrat
0,OQ,7,18,C,53.0,1,AUT
1,OQ,6,18,C,53.0,1,AUT
2,OQ,6,18,C,53.0,1,AUT
3,OQ,14,18,C,53.0,1,AUT
4,OQ,13,18,C,53.0,1,AUT
...,...,...,...,...,...,...,...
2175639,GI,13,26,C,56.0,1,CDI
2175640,OQ,12,42,C,56.0,2,CDI
2175641,GI,7,17,C,68.0,1,APP
2175642,GI,16,53,C,63.0,1,CDI


In [12]:
np.unique(df_salarie['Sexe'])

array([1, 2], dtype=int64)

In [13]:
df_salarie.loc[df_salarie["Sexe"] == 1, "Sexe"] = "Homme"
df_salarie.loc[df_salarie["Sexe"] == 2, "Sexe"] = "Femme"

In [14]:
np.unique(df_salarie['Temps_travail'])

array(['C', 'N', 'P'], dtype=object)

In [15]:
df_salarie.loc[df_salarie["Temps_travail"] == "C", "Temps_travail"] = "Temps complet"
df_salarie.loc[df_salarie["Temps_travail"] == "N", "Temps_travail"] = "Non concerné"
df_salarie.loc[df_salarie["Temps_travail"] == "P", "Temps_travail"] = "Temps partiel"

In [16]:
np.unique(df_salarie['Type_contrat'])

array(['APP', 'AUT', 'CDD', 'CDI', 'TOA', 'TTP'], dtype=object)

In [17]:
df_salarie.loc[df_salarie["Type_contrat"] == "APP", "Type_contrat"] = "Contrat d'apprentissage"
df_salarie.loc[df_salarie["Type_contrat"] == "AUT", "Type_contrat"] = "Autre type de contrat"
df_salarie.loc[df_salarie["Type_contrat"] == "TOA", "Type_contrat"] = "Travail occasionnel"
df_salarie.loc[df_salarie["Type_contrat"] == "TTP", "Type_contrat"] = "Contrat de travail temporaire"

In [18]:
np.unique(df_salarie['Activité_6'])

array(['AZ', 'BE', 'FZ', 'GI', 'JU', 'OQ'], dtype=object)

In [19]:
df_salarie.loc[df_salarie["Activité_6"] == "AZ", "Activité_6"] = "Agriculture, sylviculture et pêche"
df_salarie.loc[df_salarie["Activité_6"] == "BE", "Activité_6"] = "Industrie manufacturière, industries extractives et autres"
df_salarie.loc[df_salarie["Activité_6"] == "FZ", "Activité_6"] = "Construction"
df_salarie.loc[df_salarie["Activité_6"] == "GI", "Activité_6"] = "Commerce de gros et de détail, transports, hébergement et restauration"
df_salarie.loc[df_salarie["Activité_6"] == "JU", "Activité_6"] = "Services divers"
df_salarie.loc[df_salarie["Activité_6"] == "OQ", "Activité_6"] = "Administration publique, enseignement, santé humaine et action sociale"

In [20]:
np.unique(df_salarie['Catégorie_SP'])

array([10., 21., 22., 23., 31., 33., 34., 35., 37., 38., 42., 43., 44.,
       45., 46., 47., 48., 52., 53., 54., 55., 56., 62., 63., 64., 65.,
       67., 68., 69.])

In [21]:
df_salarie.loc[df_salarie["Catégorie_SP"] == 10.0, "Catégorie_SP"] = "Agriculteurs"
df_salarie.loc[df_salarie["Catégorie_SP"] == 21.0, "Catégorie_SP"] = "Artisans"
df_salarie.loc[df_salarie["Catégorie_SP"] == 22.0, "Catégorie_SP"] = "Commerçants et assimilés"
df_salarie.loc[df_salarie["Catégorie_SP"] == 23.0, "Catégorie_SP"] = "Chefs d'entreprise de 10 salariés ou plus"
df_salarie.loc[df_salarie["Catégorie_SP"] == 31.0, "Catégorie_SP"] = "Professions libérales"
df_salarie.loc[df_salarie["Catégorie_SP"] == 33.0, "Catégorie_SP"] = "Cadres de la fonction publique"
df_salarie.loc[df_salarie["Catégorie_SP"] == 34.0, "Catégorie_SP"] = "Professeurs, professions scientifiques"
df_salarie.loc[df_salarie["Catégorie_SP"] == 35.0, "Catégorie_SP"] = "Professions de l'information, des arts et des spectacles"
df_salarie.loc[df_salarie["Catégorie_SP"] == 37.0, "Catégorie_SP"] = "Cadres administratifs et commerciaux d'entreprises"
df_salarie.loc[df_salarie["Catégorie_SP"] == 38.0, "Catégorie_SP"] = "Ingénieurs et cadres techniques d'entreprises"
df_salarie.loc[df_salarie["Catégorie_SP"] == 42.0, "Catégorie_SP"] = "Professeurs des écoles, instituteurs et professions assimilées"
df_salarie.loc[df_salarie["Catégorie_SP"] == 43.0, "Catégorie_SP"] = "Professions intermédiaires de la santé et du travail social"
df_salarie.loc[df_salarie["Catégorie_SP"] == 44.0, "Catégorie_SP"] = "Clergé, religieux"
df_salarie.loc[df_salarie["Catégorie_SP"] == 45.0, "Catégorie_SP"] = "Professions intermédiaires administratives de la fonction publique"
df_salarie.loc[df_salarie["Catégorie_SP"] == 46.0, "Catégorie_SP"] = "Professions intermédiaires administratives et commerciales des entreprises"
df_salarie.loc[df_salarie["Catégorie_SP"] == 47.0, "Catégorie_SP"] = "Techniciens"
df_salarie.loc[df_salarie["Catégorie_SP"] == 48.0, "Catégorie_SP"] = "Contremaîtres, agents de maîtrise"
df_salarie.loc[df_salarie["Catégorie_SP"] == 52.0, "Catégorie_SP"] = "Employés civils et agents de service de la fonction publique"
df_salarie.loc[df_salarie["Catégorie_SP"] == 53.0, "Catégorie_SP"] = "Agents de surveillance"
df_salarie.loc[df_salarie["Catégorie_SP"] == 54.0, "Catégorie_SP"] = "Employés administratifs d'entreprise"
df_salarie.loc[df_salarie["Catégorie_SP"] == 55.0, "Catégorie_SP"] = "Employés de commerce"
df_salarie.loc[df_salarie["Catégorie_SP"] == 56.0, "Catégorie_SP"] = "Personnels des services directs aux particuliers"
df_salarie.loc[df_salarie["Catégorie_SP"] == 62.0, "Catégorie_SP"] = "Ouvriers qualifiés de type industriel"
df_salarie.loc[df_salarie["Catégorie_SP"] == 63.0, "Catégorie_SP"] = "Ouvriers qualifiés de type artisanal"
df_salarie.loc[df_salarie["Catégorie_SP"] == 64.0, "Catégorie_SP"] = "Chauffeurs"
df_salarie.loc[df_salarie["Catégorie_SP"] == 65.0, "Catégorie_SP"] = "Ouvriers qualifiés de la manutention,  du magasinage et du transport"
df_salarie.loc[df_salarie["Catégorie_SP"] == 67.0, "Catégorie_SP"] = "Ouvriers non qualifiés de type industriel"
df_salarie.loc[df_salarie["Catégorie_SP"] == 68.0, "Catégorie_SP"] = "Ouvriers non qualifiés de type artisanal"
df_salarie.loc[df_salarie["Catégorie_SP"] == 69.0, "Catégorie_SP"] = "Ouvriers agricoles et assimilés"

In [22]:
df_salarie.to_csv('df_salarie_full.csv', index = False)

In [28]:
df_salarie.sample(5)

,Activité_6,Salaire_net,Age,Temps_travail,Catégorie_SP,Sexe,Type_contrat
1856307,"Administration publique, enseignement, santé h...",17,32,Temps complet,Professions intermédiaires de la santé et du t...,Femme,Autre type de contrat
1257331,"Commerce de gros et de détail, transports, héb...",9,28,Temps partiel,Employés de commerce,Femme,CDI
284457,"Administration publique, enseignement, santé h...",15,47,Temps partiel,Employés civils et agents de service de la fon...,Femme,Autre type de contrat
1661195,"Commerce de gros et de détail, transports, héb...",22,48,Temps complet,Cadres de la fonction publique,Homme,CDI
1116431,"Administration publique, enseignement, santé h...",13,58,Temps complet,Ouvriers qualifiés de type industriel,Femme,Autre type de contrat


In [24]:
df_salarie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2175644 entries, 0 to 2175643
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Activité_6     object
 1   Salaire_net    int64 
 2   Age            int64 
 3   Temps_travail  object
 4   Catégorie_SP   object
 5   Sexe           object
 6   Type_contrat   object
dtypes: int64(2), object(5)
memory usage: 116.2+ MB


In [26]:
df_salarie.describe().round(2)

,Salaire_net,Age
count,2175644.00,2175644.00
mean,14.63,40.09
std,4.88,12.36
min,0.00,16.00
25%,11.00,30.00
50%,14.00,40.00
75%,18.00,50.00
max,23.00,69.00
